##### Copyright 2024 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Audio Quickstart

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Audio.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

This notebook provides an example of how to prompt Gemini 1.5 Pro using an audio file. In this case, you'll use a [sound recording](https://www.jfklibrary.org/asset-viewer/archives/jfkwha-006) of President John F. Kennedy’s 1961 State of the Union address.

In [ ]:
!pip install -q -U google-generativeai

In [ ]:
import google.generativeai as genai

## Configure your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) for an example.

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

## Upload an audio file with the File API

To use an audio file in your prompt, you must first upload it using the [File API](https://github.com/google-gemini/cookbook/blob/main/quickstarts/File_API.ipynb).


In [ ]:
URL = "https://storage.googleapis.com/generativeai-downloads/data/State_of_the_Union_Address_30_January_1961.mp3"

In [ ]:
!wget -q $URL -O sample.mp3

In [ ]:
your_file = genai.upload_file(path='sample.mp3')

## Use the file in your prompt

In [ ]:
prompt = "Listen carefully to the following audio file. Provide a brief summary."
model = genai.GenerativeModel('models/gemini-1.5-pro-latest')
response = model.generate_content([prompt, your_file])
print(response.text)

## Summary of President John F. Kennedy's 1961 State of the Union Address:

**Main Theme:** The address focuses on the challenges and opportunities facing the United States both domestically and internationally, emphasizing the need for unity and action in the face of the Cold War and economic difficulties.

**Key Points:**

* **Economic Concerns:** Kennedy highlights the country's economic troubles, including a recession, high unemployment, and falling farm income. He proposes measures to address these issues, such as increased unemployment compensation, minimum wage increases, and tax incentives for investment.
* **Balance of Payments Deficit:** Kennedy acknowledges the growing deficit but assures the nation that the dollar remains strong and pledges not to devalue it. He outlines steps to attract foreign investment, promote exports, and curb spending abroad.
* **Unfinished Domestic Tasks:** Kennedy addresses several domestic issues needing attention, including urban decay, education

## Count audio tokens

You can count the number of tokens in your audio file like this.

In [ ]:
model.count_tokens([your_file])

total_tokens: 78330

## Learning more

* Learn more about the [File API](https://github.com/google-gemini/cookbook/blob/main/quickstarts/File_API.ipynb) with the quickstart.

* Learn more about prompting with [media files](https://ai.google.dev/tutorials/prompting_with_media) in the docs, including the supported formats and maximum length for audio files.